In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Create data

In [2]:
import copy
from liegroups import SE2, SO2

params_true = {'T_1_0': SE2.identity(),
               'T_2_0': SE2(SO2.identity(), -np.array([0.5, 0])),
               'T_3_0': SE2(SO2.identity(), -np.array([1, 0])),
               'T_4_0': SE2(SO2.fromangle(np.pi / 2),
                             -(SO2.fromangle(np.pi / 2) * np.array([1, 0.5]))),
               'T_5_0': SE2(SO2.fromangle(np.pi),
                             -(SO2.fromangle(np.pi) * np.array([0.5, 0.5]))),
               'T_6_0': SE2(SO2.fromangle(-np.pi / 2),
                             -(SO2.fromangle(-np.pi / 2) * np.array([0.5, 0])))}

obs = {'T_1_0': params_true['T_1_0'],
       'T_2_1': params_true['T_2_0'] * params_true['T_1_0'].inv(),
       'T_3_2': params_true['T_3_0'] * params_true['T_2_0'].inv(),
       'T_4_3': params_true['T_4_0'] * params_true['T_3_0'].inv(),
       'T_5_4': params_true['T_5_0'] * params_true['T_4_0'].inv(),
       'T_6_5': params_true['T_6_0'] * params_true['T_5_0'].inv(),
       'T_6_2': params_true['T_6_0'] * params_true['T_2_0'].inv()}

params_init = copy.deepcopy(params_true)
for key in params_init.keys():
    params_init[key] = SE2.exp(5 * np.random.rand(3)) * params_init[key]

# Create cost functions

In [4]:
from pyslam.costs import PoseCost, PoseToPoseCost
from pyslam.utils import invsqrt

prior_stiffness = invsqrt(1e-12 * np.identity(3))
odom_stiffness = invsqrt(1e-3 * np.identity(3))
loop_stiffness = invsqrt(1e-3 * np.identity(3))

cost0 = PoseCost(obs['T_1_0'], prior_stiffness)
cost0_params = ['T_1_0']

cost1 = PoseToPoseCost(obs['T_2_1'], odom_stiffness)
cost1_params = ['T_1_0', 'T_2_0']

cost2 = PoseToPoseCost(obs['T_3_2'], odom_stiffness)
cost2_params = ['T_2_0', 'T_3_0']

cost3 = PoseToPoseCost(obs['T_4_3'], odom_stiffness)
cost3_params = ['T_3_0', 'T_4_0']

cost4 = PoseToPoseCost(obs['T_5_4'], odom_stiffness)
cost4_params = ['T_4_0', 'T_5_0']

cost5 = PoseToPoseCost(obs['T_6_5'], odom_stiffness)
cost5_params = ['T_5_0', 'T_6_0']

cost6 = PoseToPoseCost(obs['T_6_2'], loop_stiffness)
cost6_params = ['T_2_0', 'T_6_0']

AttributeError: module 'scipy' has no attribute 'linalg'

# Set up and solve the problem

In [ ]:
from pyslam.problem import Problem, Options

options = Options()
options.allow_nondecreasing_steps = True
options.max_nondecreasing_steps = 3
options.print_iter_summary = True

problem = Problem(options)

problem.add_residual_block(cost0, cost0_params)
problem.add_residual_block(cost1, cost1_params)
problem.add_residual_block(cost2, cost2_params)
problem.add_residual_block(cost3, cost3_params)
problem.add_residual_block(cost4, cost4_params)
problem.add_residual_block(cost5, cost5_params)
# problem.add_residual_block(cost6, cost6_params)

problem.initialize_params(params_init)

params_final = problem.solve()

# Check results

In [ ]:
print("Initial Error:")
for key in params_true.keys():
    print('{}: {}'.format(key, SE2.log(params_init[key].inv() * params_true[key])))

print()

print("Final Error:")
for key in params_true.keys():
    print('{}: {}'.format(key, SE2.log(params_final[key].inv() * params_true[key])))

# Optional: Compute the covariance of the final parameter estimates

In [ ]:
problem.compute_covariance()
print('covariance of T_5_0:\n{}'.format( problem.get_covariance_block('T_5_0','T_5_0') ))